In [1]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [2]:
data_fake=pd.read_csv('Fake.csv')
data_true=pd.read_csv('True.csv')


In [3]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
data_fake["class"]=0
data_true["class"]=1


In [6]:
data_fake.shape,data_true.shape

((23481, 5), (21417, 5))

In [7]:
data_fake_manual_testing=data_fake.tail(10)
for i in range (23480,23470,-1):
    data_fake.drop([i],axis=0,inplace=True)
    
data_true_manual_testing=data_true.tail(10)
for i in range (21416,21406,-1):
    data_true.drop([i] , axis=0, inplace=True)

In [8]:
df_manual_testing=pd.concat([data_fake_manual_testing,data_true_manual_testing],axis=0)
df_manual_testing.to_csv("manual_testing.csv")

In [9]:
df_merge=pd.concat([data_fake,data_true],axis=0)
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [10]:
df=df_merge.drop(["title","subject","date"],axis=1)
df.head(10)

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
5,The number of cases of cops brutalizing and ki...,0
6,Donald Trump spent a good portion of his day a...,0
7,In the wake of yet another court decision that...,0
8,Many people have raised the alarm regarding th...,0
9,Just when you might have thought we d get a br...,0


In [11]:
df=df.sample(frac=1)

In [12]:
df.head(10)

,text,class
18370,"On April 3, 2017, Brooklyn-born Sarsour, daugh...",0
9208,The Deputy Attorney General Rod Rosenstein tes...,0
12456,They pulled every trick in the book. From manu...,0
5556,WASHINGTON (Reuters) - (This February 8 story ...,1
18726,"Take note cowardly anti-American thugs, your B...",0
6456,DETROIT (Reuters) - Fiat Chrysler Automobiles ...,1
795,WASHINGTON (Reuters) - U.S. Commerce Secretary...,1
7307,WASHINGTON (Reuters) - Congressional Republica...,1
19406,MEXICO CITY (Reuters) - Citigroup Inc s local ...,1
6640,MOSCOW (Reuters) - The Kremlin is not in touch...,1


In [13]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [14]:
def word_drop(text):
  text= text.lower()
  text=re.sub('\[.*?\]','',text)
  text=re.sub("\\W"," ", text)
  text=re.sub('https?://\S+|www\.\S+','',text)
  text=re.sub('<.*?>+','',text)
  text=re.sub('[%s]' % re.escape(string.punctuation),'',text)
  text=re.sub('\n','',text)
  text=re.sub('\w*\d\w*','',text)
  return text

In [15]:
df["text"]=df["text"].apply(word_drop)

In [16]:
df.head(10)

,text,class
18370,on april brooklyn born sarsour daughter o...,0
9208,the deputy attorney general rod rosenstein tes...,0
12456,they pulled every trick in the book from manu...,0
5556,washington reuters this february story h...,1
18726,take note cowardly anti american thugs your b...,0
6456,detroit reuters fiat chrysler automobiles ...,1
795,washington reuters u s commerce secretary...,1
7307,washington reuters congressional republica...,1
19406,mexico city reuters citigroup inc s local ...,1
6640,moscow reuters the kremlin is not in touch...,1


In [17]:
x=df["text"]
y=df["class"]


In [18]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
vectorization=TfidfVectorizer()
xv_train=vectorization.fit_transform(x_train)
xv_test=vectorization.transform(x_test)

Logistic regression

In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
LR=LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [23]:
LR.score(xv_test,y_test)

0.9881461675579323

In [24]:
pred_LR=LR.predict(xv_test)

In [25]:
print(classification_report(y_test,pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5781
           1       0.99      0.99      0.99      5439

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



Decision Tree classification

In [26]:
from sklearn.tree import DecisionTreeClassifier


In [27]:
DT=DecisionTreeClassifier()
DT.fit(xv_train,y_train)

DecisionTreeClassifier()

In [28]:
DT.score(xv_test,y_test)

0.9957219251336898

In [29]:
pred_DT=DT.predict(xv_test)

In [30]:
print(classification_report(y_test,pred_DT))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5781
           1       1.00      1.00      1.00      5439

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



Gradient Boosting Classifier

In [31]:
from sklearn.ensemble import GradientBoostingClassifier

In [32]:
GBC=GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train,y_train)

GradientBoostingClassifier(random_state=0)

In [33]:
GBC.score(xv_test,y_test)

0.9962566844919786

In [34]:
pred_GBC=GBC.predict(xv_test)

In [35]:
print(classification_report(y_test,pred_GBC))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5781
           1       0.99      1.00      1.00      5439

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



Random Forest Classifier

In [36]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
RFC=RandomForestClassifier(random_state=0)
RFC.fit(xv_train,y_train)

RandomForestClassifier(random_state=0)

In [38]:
RFC.score(xv_test,y_test)

0.990909090909091

In [39]:
pred_RFC=RFC.predict(xv_test)

In [40]:
print(classification_report(y_test,pred_RFC))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5781
           1       0.99      0.99      0.99      5439

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



Manual Testing 

In [48]:
def output_label(n):
    if n==0:
        return "Fake News"
    elif n==1:
        return "NOt A Fake News"
    
def manual_testing(news):
    testing_news={"text":[news]}
    new_def_test=pd.DataFrame(testing_news)
    new_def_test["text"]=new_def_test["text"].apply(word_drop)
    new_x_test=new_def_test["text"]
    new_xv_test=vectorization.transform(new_x_test)
    pred_LR=LR.predict(new_xv_test)
    pred_DT=DT.predict(new_xv_test)
    
    return print("\n\n'LR Prediction:{}' \n'DT Prediction: {}' \n'GBC Prediction: {}' \n'RFC Prediction: {}'".format(output_label(pred_LR[0]),
                        output_label(pred_GBC[0]),
                        output_label(pred_RFC[0])))
    

In [50]:
news=str(input())
manual_testing(news)

IndexError: Replacement index 3 out of range for positional args tuple